In [1]:
from fastapi import FastAPI, HTTPException
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
from typing import List, Dict, Any

#hot_df = pd.read_csv("./data/hot_results_20240113231450.csv")
#hot_df

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('./data/hot_data.db')
text_data = pd.read_sql('SELECT * FROM sentences', conn)

labels = ['neutral', 'surprise', 'sadness', 'joy', 'fear', 'disgust', 'anger']
neutral = ['neutral']
negative = ['sadness', 'fear', 'disgust', 'anger']
positive = ['joy', 'surprise']

def get_label(sentiment):
	if sentiment in neutral:
		return 'neutral'
	elif sentiment in negative:
		return 'negative'
	elif sentiment in positive:
		return 'positive'
	else:
		return 'unknown'

text_data['label'] = text_data['emotion_label'].apply(get_label)
text_data.to_sql('sentences', conn, if_exists='replace', index=False)

text_data = pd.read_sql('SELECT * FROM sentences', conn)
text_data

,id,emotion_label,emotion_score,text,label
0,f0016236-f87f-4ab4-8689-42ce66fa6295,joy,0.770712,Hey Slugs!,positive
1,67e23947-8c13-4427-ad59-ae1398f2f225,fear,0.786226,Be aware of common craigslist housing scams!,negative
2,77d4e3cb-b557-47e7-bd79-39cee78d3e02,neutral,0.577710,"Of course not all post are scams, but you shou...",neutral
3,def31337-ba79-49a8-8b07-910b355aaa56,neutral,0.802858,1.,neutral
4,2bfb56cf-db29-4ab8-87d6-db696fda292c,neutral,0.643948,The Listing Has No Photos\n2.,neutral
...,...,...,...,...,...
17511,b682018a-ce03-461b-9853-d6130e91dbb0,neutral,0.962796,The underline Unix system on Macs are just far...,neutral
17512,fa4d2c10-d175-4071-b137-e21ab1fba328,neutral,0.731793,>Like why wouldn’t u just start out with a win...,neutral
17513,6e81543b-d7a4-4dc5-bbdf-02bc7324cddf,disgust,0.369341,Windows computers are much more annoying to us...,negative
17514,b4347747-ebe5-4593-9d62-4c25511a1c12,neutral,0.516326,"CS student, but windows surface has been doing...",neutral


In [3]:
text_data


,id,emotion_label,emotion_score,text,label
0,f0016236-f87f-4ab4-8689-42ce66fa6295,joy,0.770712,Hey Slugs!,positive
1,67e23947-8c13-4427-ad59-ae1398f2f225,fear,0.786226,Be aware of common craigslist housing scams!,negative
2,77d4e3cb-b557-47e7-bd79-39cee78d3e02,neutral,0.577710,"Of course not all post are scams, but you shou...",neutral
3,def31337-ba79-49a8-8b07-910b355aaa56,neutral,0.802858,1.,neutral
4,2bfb56cf-db29-4ab8-87d6-db696fda292c,neutral,0.643948,The Listing Has No Photos\n2.,neutral
...,...,...,...,...,...
17511,b682018a-ce03-461b-9853-d6130e91dbb0,neutral,0.962796,The underline Unix system on Macs are just far...,neutral
17512,fa4d2c10-d175-4071-b137-e21ab1fba328,neutral,0.731793,>Like why wouldn’t u just start out with a win...,neutral
17513,6e81543b-d7a4-4dc5-bbdf-02bc7324cddf,disgust,0.369341,Windows computers are much more annoying to us...,negative
17514,b4347747-ebe5-4593-9d62-4c25511a1c12,neutral,0.516326,"CS student, but windows surface has been doing...",neutral


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
nltk.download('stopwords')

class TextAnalyzer:
	def __init__(self, data_file):
		#self.data = pd.read_csv(data_file)
		conn = sqlite3.connect('./data/hot_data.db')
		self.data = pd.read_sql('SELECT * FROM sentences', conn)
		self.labels = ['neutral', 'surprise', 'sadness', 'joy', 'fear', 'disgust', 'anger']
		self.categories = {
			'all': self.labels,
			'neutral': ['neutral'],
			'negative': ['sadness', 'fear', 'disgust', 'anger'],
			'positive': ['joy', 'surprise']
		}
		self.label_word_freq = {}
		self.label_word_tfidf = {}

	def get_word_frequencies(self, text_data):
		vectorizer = CountVectorizer(stop_words='english')
		word_count = vectorizer.fit_transform(text_data)
		sum_words = word_count.sum(axis=0)
		word_freq = [(word, int(sum_words[0, idx])) for word, idx in vectorizer.vocabulary_.items()]
		return sorted(word_freq, key=lambda x: x[1], reverse=True)

	def get_tfidf_word_frequencies(self, text_data):
		vectorizer = TfidfVectorizer(stop_words='english')
		tfidf_matrix = vectorizer.fit_transform(text_data.dropna())
		feature_names = vectorizer.get_feature_names_out()
		dense = tfidf_matrix.todense()
		denselist = dense.tolist()
		df = pd.DataFrame(denselist, columns=feature_names)
		return df.mean(axis=0).sort_values(ascending=False).reset_index().rename(columns={0: 'score'})

	def analyze(self):
		for key in self.categories.keys():
			filtered_data = self.data[self.data['emotion_label'].isin(self.categories[key])]['text']
			self.label_word_freq[key] = self.get_word_frequencies(filtered_data)
			#self.label_word_tfidf[key] = self.get_tfidf_word_frequencies(filtered_data)

		print("Done loading data")

#if __name__ == "__main__":
#	text_analyzer = TextAnalyzer("data/hot_results.csv")
#	text_analyzer.analyze()

#	print("Word frequencies:")
#	print(text_analyzer.label_word_freq)
	


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/masaishi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
text_analyzer = TextAnalyzer("data/hot_results.csv")
text_analyzer.analyze()

print("Word frequencies:")
print(text_analyzer.label_word_freq)

Done loading data
Word frequencies:
{'all': [('just', 1152), ('like', 921), ('people', 864), ('don', 823), ('campus', 764), ('ucsc', 749), ('know', 605), ('time', 567), ('year', 567), ('class', 558), ('looking', 513), ('think', 489), ('good', 443), ('students', 434), ('want', 423), ('housing', 420), ('ve', 408), ('school', 401), ('https', 390), ('really', 383), ('ll', 380), ('need', 369), ('classes', 362), ('make', 338), ('student', 326), ('major', 325), ('going', 316), ('work', 309), ('pretty', 307), ('did', 301), ('got', 297), ('room', 296), ('lot', 282), ('sure', 277), ('way', 277), ('quarter', 257), ('say', 256), ('college', 247), ('person', 247), ('right', 246), ('bus', 236), ('didn', 231), ('let', 229), ('doesn', 225), ('years', 224), ('probably', 223), ('place', 219), ('santa', 207), ('new', 207), ('lol', 206), ('cruz', 204), ('use', 202), ('rent', 199), ('better', 193), ('feel', 191), ('job', 189), ('yeah', 188), ('getting', 183), ('help', 183), ('single', 182), ('thing', 179),

In [30]:
words_dict = {}
for item in text_analyzer.label_word_freq['all']:
	words_dict[item[0]] = {}
for category in text_analyzer.label_word_freq.keys():
	for item in text_analyzer.label_word_freq[category]:
		if item[0] in words_dict.keys():
			words_dict[item[0]][f'{category}_freq'] = item[1]
words_dict

{'just': {'all_freq': 1152,
  'neutral_freq': 814,
  'negative_freq': 203,
  'positive_freq': 135},
 'like': {'all_freq': 921,
  'neutral_freq': 566,
  'negative_freq': 213,
  'positive_freq': 142},
 'people': {'all_freq': 864,
  'neutral_freq': 517,
  'negative_freq': 234,
  'positive_freq': 113},
 'don': {'all_freq': 823,
  'neutral_freq': 514,
  'negative_freq': 219,
  'positive_freq': 90},
 'campus': {'all_freq': 764,
  'neutral_freq': 533,
  'negative_freq': 106,
  'positive_freq': 125},
 'ucsc': {'all_freq': 749,
  'neutral_freq': 545,
  'negative_freq': 110,
  'positive_freq': 94},
 'know': {'all_freq': 605,
  'neutral_freq': 435,
  'negative_freq': 77,
  'positive_freq': 93},
 'time': {'all_freq': 567,
  'neutral_freq': 378,
  'negative_freq': 116,
  'positive_freq': 73},
 'year': {'all_freq': 567,
  'neutral_freq': 307,
  'negative_freq': 95,
  'positive_freq': 165},
 'class': {'all_freq': 558,
  'neutral_freq': 372,
  'negative_freq': 91,
  'positive_freq': 95},
 'looking': {

In [34]:
words_df = pd.DataFrame.from_dict(words_dict, orient='index')
words_df.index.names = ['word']
words_df.to_sql('words', conn, if_exists='replace', index=True)
words_df = pd.read_sql('SELECT * FROM words', conn)
words_df

,word,all_freq,neutral_freq,negative_freq,positive_freq
0,just,1152,814.0,203.0,135.0
1,like,921,566.0,213.0,142.0
2,people,864,517.0,234.0,113.0
3,don,823,514.0,219.0,90.0
4,campus,764,533.0,106.0,125.0
...,...,...,...,...,...
13579,dell,1,1.0,NaN,NaN
13580,proofing,1,1.0,NaN,NaN
13581,mplabx,1,1.0,NaN,NaN
13582,underline,1,1.0,NaN,NaN


In [36]:
# loop of words_df
sentence_df = pd.read_sql('SELECT * FROM sentences', conn)

for index, row in words_df.iterrows():
	result = sentence_df[sentence_df["text"].str.contains(row['word'], na=False)]
	result = result['label'].value_counts().to_dict()
	words_df.loc[index, 'neutral_num'] = result.get('neutral', 0)
	words_df.loc[index, 'positive_num'] = result.get('positive', 0)
	words_df.loc[index, 'negative_num'] = result.get('negative', 0)

words_df
	

,word,all_freq,neutral_freq,negative_freq,positive_freq,neutral_num,positive_num,negative_num
0,just,1152,814.0,203.0,135.0,687.0,122.0,194.0
1,like,921,566.0,213.0,142.0,597.0,145.0,209.0
2,people,864,517.0,234.0,113.0,454.0,107.0,207.0
3,don,823,514.0,219.0,90.0,554.0,103.0,212.0
4,campus,764,533.0,106.0,125.0,481.0,117.0,99.0
...,...,...,...,...,...,...,...,...
13579,dell,1,1.0,NaN,NaN,0.0,0.0,0.0
13580,proofing,1,1.0,NaN,NaN,1.0,0.0,0.0
13581,mplabx,1,1.0,NaN,NaN,0.0,0.0,0.0
13582,underline,1,1.0,NaN,NaN,1.0,0.0,0.0


In [37]:
#words_df.drop(columns=['level_0'], inplace=True)
#words_df

In [38]:
## Delete words table
cursor = conn.cursor()
cursor.execute("DROP TABLE words")
conn.commit()

words_df.to_sql('words', conn, if_exists='replace', index=True)
words_df = pd.read_sql('SELECT * FROM words', conn)

In [5]:
#from sklearn.feature_extraction.text import TfidfVectorizer

#def get_word_frequencies_tfidf(text_data):
#    vectorizer = TfidfVectorizer(stop_words='english')
#    tfidf_matrix = vectorizer.fit_transform(text_data)
#    sum_tfidf = tfidf_matrix.sum(axis=0)
#    word_freq = [(word, sum_tfidf[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
#    word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
#    word_freq = [{"word": word, "tfidf": count} for word, count in word_freq]

#    return word_freq

In [20]:
# Assuming `hot_df` is your DataFrame and it has two columns: 'text' and 'label'

labels = ['neutral', 'surprise', 'sadness', 'joy', 'fear', 'disgust', 'anger']
neutral = ['neutral']
negative = ['sadness', 'fear', 'disgust', 'anger']
positive = ['joy', 'surprise']
categorries = {
	'neutral': neutral,
	'negative': negative,
	'positive': positive
}

label_word_freq = {}

for key in categorries.keys():
	filtered_data = hot_df[hot_df['label'].isin(categorries[key])]['text']
	#word_freq = get_word_frequencies_tfidf(filtered_data)
	label_word_freq[key] = word_freq

# label_word_freq now contains word frequencies for each label


In [45]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
	model="gpt-3.5-turbo",
	messages=[
		{"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
		{"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
	]
)

print(completion.choices[0].message)

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.

In [47]:
from fastapi import FastAPI, HTTPException, Query
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from typing import List, Dict, Any


# Create an SQLAlchemy engine
engine = create_engine('sqlite:///./data/hot_data.db')

# Create a Session local class
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

categories = ["all", "neutral", "positive", "negative"]

def word_freq(
    limit: int = 50,
    examples_limit: int = 5,
    label: str = Query("all", enum=categories, description="Label to filter by"),
) -> List[Dict[str, Any]]:
    freq_column = f"{label}_freq"

    # SQL query to retrieve words and their frequencies
    word_query = text(f"""
    SELECT `index`, `{freq_column}`
    FROM words
    WHERE `{freq_column}` IS NOT NULL
    ORDER BY `{freq_column}` DESC
    LIMIT {limit}
    """)

    # Prepare the output
    output = []

    with SessionLocal() as session:
        words = session.execute(word_query).fetchall()

        for i, (word, freq) in enumerate(words):
            # SQL query to retrieve example sentences for the word
            sentence_query = text(f"""
            SELECT *
            FROM sentences
            WHERE text LIKE '%{word}%'
            LIMIT {examples_limit}
            """)

            example_sentences = session.execute(sentence_query).fetchall()
            examples = [sentence for sentence in example_sentences]
            print(examples)

            output.append({
                "word": word,
                "frequency": freq,
                "examples": examples
            })

            # Limit the number of words returned
            if i >= examples_limit - 1:
                break

    return output


word_freq()

OperationalError: (sqlite3.OperationalError) no such column: annotation=NoneType required=False default='all' description='Label to filter by' json_schema_extra={'enum': ['all', 'neutral', 'positive', 'negative']}_freq
[SQL: 
    SELECT `index`, `annotation=NoneType required=False default='all' description='Label to filter by' json_schema_extra={'enum': ['all', 'neutral', 'positive', 'negative']}_freq`
    FROM words
    WHERE `annotation=NoneType required=False default='all' description='Label to filter by' json_schema_extra={'enum': ['all', 'neutral', 'positive', 'negative']}_freq` IS NOT NULL
    ORDER BY `annotation=NoneType required=False default='all' description='Label to filter by' json_schema_extra={'enum': ['all', 'neutral', 'positive', 'negative']}_freq` DESC
    LIMIT 50
    ]
(Background on this error at: https://sqlalche.me/e/20/e3q8)